In [3]:
import numpy as np
import matplotlib.pyplot as plt
import iop_ext as iop
import lf14 as lf
import math

In [20]:
#assigning constants

#use NEA & use constants regarding a singular exoplanet
au=1.49598e11 #meters - astronomical unit(dist. from Earth to Sun)
L_sun=3.828e26 #Watts - luminosity of our star
M_sun=1.989e30 #Kg - mass of our star
M_ea=5.97219e24 #Kg - mass of earth
R_ea=6.371e6 #meters - radius of earth
L_star=0.5*L_sun #placeholder 
M_star=0.665*M_sun
Teq=871 # Equilibrium temperature in Kelvin
a= 0.0422*au #semimajor axis
a_0 = 0.5 ##slope at which L_He decreases over time
yr=3600*24*365 #Converting year to seconds
t_sat=1e8*yr #100 million years to seconds
cmf_p=0.325 #reduced core mass fraction
G=6.67e-11 #Universal Gravitational Constant
L_sat=(10**-3.5)*L_sun*(M_star/M_sun) #Saturation Luminosity
pi=math.pi
x0=0.5 #initial water mass fraction
Gyr=1e9 #billion years



In [5]:
#initalizing variables
t=1e6*yr #1 million years to seconds
#Range from 1-20

In [6]:
#functions
#Mp will range from 0-20 in terms of Earth masses

#FOR RADIUS, USE IOP & ERROR IOP
def wmfraction(Mp,Mp0,X0):
    return(1-((Mp0-(X0*Mp0))/Mp))
def Rp_function(CMF,X,Teq,Mp): #Mp in kg, returns Rp in meters
    Mp_e=Mp/M_ea
    Rp_e=iop.radius_iop(CMF,X,Teq,Mp_e)
    return(Rp_e*R_ea)#X is water mass fraction, function not made yet
#High energy luminosity
def L_He_function(M_star,t): #t in Myr
    if t<t_sat:
        return(L_sat)
    else:
        return((L_sat)*((t/t_sat)**(-1-a_0)))
#Efficiency parameter η - how much HE is converted into mass loss
def eff_par(Mp,Rp):
    v_esc=((2*(G)*Mp)/(Rp))**(0.5)
    return(0.1*(((v_esc)/(15))**(-2)))
#Mass-loss rate
def ml_rate(Mp,Rp,a,t): #In earth masses/Billion Years
    L_HiE=L_He_function(M_star,t)
    e_p=eff_par(Mp,Rp)
    return(e_p*((Rp**3)*L_HiE)/(4*(a**2)*(G)*Mp)) #given in kg/s

    

In [7]:
Rp=Rp_function(0.325,0,300,1)
print(Rp)

2876947.206561057


In [8]:
#function testing
print(wmfraction(0.66*M_ea,1*M_ea,x0))
print(L_He_function(M_sun,1e9*yr))
print(eff_par(M_ea,R_ea))
print(ml_rate(M_ea,R_ea,a,t)*(1e9*yr)/M_ea)   #Earth masses per Gyr
print(ml_rate(M_ea,R_ea,a,t)) #kg/s

0.24242424242424243
2.5456200000000004e+21
1.7992879756291102e-07
3.114541630847878e-07
58.98222470298513


In [25]:
#Evolution
t_evo=1e6*yr #time
del_t=1e6*yr #increment of time changing per 1 million years
tmax=5e9*yr #placeholder, to prevent computer overload
Mp_0=4.53*M_ea #original mass of planet
X_0=0.5 #original WMF
Rp_evo=1.704*R_ea #radius of the planet that will change over time
Mp_evo=4.53*M_ea #mass of the planet that will change over time
X_evo=0.5 #WMF that will change over time
ml=ml_rate(Mp_evo,Rp_evo,a,t_evo)
error_code=iop.error_iop(cmf_p,X_evo,Teq,Mp_evo/M_ea)
while X_evo>=0: 
    M_multi=((ml*del_t))
    Mp_evo=(Mp_evo-M_multi)
    X_evo=wmfraction(Mp_evo,Mp_0,x0)
    Rp_evo=(Rp_function(cmf_p,X_evo,Teq,Mp_evo))
    error_code=iop.error_iop(cmf_p,X_evo,Teq,Mp_evo/M_ea)
    t_evo=t_evo+del_t #increasing time
    
    if t_evo>tmax:
        print("time")
        break
    elif Rp_evo<0:
        print("negative radius")
        break
    elif ml<0:
        print("Negative ml rate")
        break
    elif error_code==2:
        print("error code = 2")
        break
    
    ml=ml_rate(Mp_evo,Rp_evo,a,t_evo)
    
    print(f"t: {(t_evo/(1e9*yr)):.4e}, mdot: {(ml*1e9*yr)/M_ea:0.10f}, mp: {(Mp_evo/M_ea):0.9e} , rp: {(Rp_evo/R_ea):0.12e}, wmf: {X_evo:0.12f} , error:{error_code:0.2f}")
        
   

t: 2.0000e-03, mdot: 0.0000005023, mp: 4.530000000e+00 , rp: 2.398553036685e+00, wmf: 0.499999999986 , error:0.00
t: 3.0000e-03, mdot: 0.0000005023, mp: 4.529999999e+00 , rp: 2.398553036593e+00, wmf: 0.499999999930 , error:0.00
t: 4.0000e-03, mdot: 0.0000005023, mp: 4.529999999e+00 , rp: 2.398553036502e+00, wmf: 0.499999999875 , error:0.00
t: 5.0000e-03, mdot: 0.0000005023, mp: 4.529999998e+00 , rp: 2.398553036411e+00, wmf: 0.499999999820 , error:0.00
t: 6.0000e-03, mdot: 0.0000005023, mp: 4.529999998e+00 , rp: 2.398553036319e+00, wmf: 0.499999999764 , error:0.00
t: 7.0000e-03, mdot: 0.0000005023, mp: 4.529999997e+00 , rp: 2.398553036228e+00, wmf: 0.499999999709 , error:0.00
t: 8.0000e-03, mdot: 0.0000005023, mp: 4.529999997e+00 , rp: 2.398553036136e+00, wmf: 0.499999999653 , error:0.00
t: 9.0000e-03, mdot: 0.0000005023, mp: 4.529999996e+00 , rp: 2.398553036045e+00, wmf: 0.499999999598 , error:0.00
t: 1.0000e-02, mdot: 0.0000005023, mp: 4.529999996e+00 , rp: 2.398553035953e+00, wmf: 0.

In [11]:
#deltat should be 1 million years